# Project 3: Analyzing the behaviour of the word "anymore" in different geographical regions
## Sid's Notebook

# Scraping Comments from Reddit using Praw 

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np

# Start your instance of reddit API

In [2]:
## start your reddit API
import praw
instance=praw.Reddit(client_id='####',
                     client_secret='###', 
                     username='###', 
                     password='###', 
                     user_agent='sidvash')

In [3]:
list_subreds = ['nyc', 'seattle', 'chicago', 'toronto', 'losangeles', 'portland', 
                'boston', 'austin', 'sanfrancisco', 'vancouver', 'washingtondc',
               'london']

# Retrieve titles, comments and replies from Reddit threads


In [16]:
def scrape_reddit(subred, word, cols):
    '''
    Input: 
        1. Subreddit_list: A list of subreddits to be scraped
        2. word: only comments/threads containing this word would be scraped
    Output:
        1. A dict with all stored commments
        2. Output a Pandas dataframes containi
    
    
    '''

    subInstance=instance.subreddit(subred)
    hotThreads=subInstance.hot(limit=2000) ## two stickies threads, skip them and only work with top hot thread

    dict_thread = {}
    dict_comment = defaultdict(list)
    count_thread=0
    count_word =0
    dataframes = []

    print("##### Processing Subreddit : {} ####\n".format(subred))

    for thread in hotThreads:
        if not thread.stickied: ## remove stickied threads
            #print(thread.title)
            if count_thread%20 == 0:
                print("Threads scraped so far: {}".format(count_thread))
                print("Count of '{}' so far: {}\n".format(word, count_word))

            #Add titles to dict_thread
            if word in thread.title.lower():
                dict_thread[count_thread] = thread.title
                count_word+=1

            thread.comments.replace_more(limit=20) ## remove all MoreComments objects

            #Add comments to the dict_comment
            for comment in thread.comments.list():
                if word in comment.body.lower():
                    dict_comment[count_thread].append(comment.body)
                    count_word +=1

            count_thread+=1
        
    data = create_dataframe(dict_thread, dict_comment, subred, cols)
        
    data.to_csv("comments_data_"+ subred + ".csv")
        
    return data
    
                   
def create_dataframe(dict_thread, dict_comment, subr, cols, index_range = 1000):
    '''
    Creates a Pandas dataframe from the given scraped data dictionary 
    '''
    data = pd.DataFrame(index = range(index_range), columns=cols)
    
    row = 0

    for thread_id, thread_title in dict_thread.items():
        data.loc[row, "subreddit"] = subr
        data.loc[row, "thread_id"] = thread_id
        data.loc[row, "body"] = thread_title
        data.loc[row, "comment_yn"] = 0
        data.loc[row, "thread_title_yn"] = 1

        row+=1

    #Store comments
    for thread_id, comments in dict_comment.items():
        for comment in comments:
            data.loc[row, "subreddit"] = subr
            data.loc[row, "thread_id"] = thread_id
            data.loc[row, "body"] = comment
            data.loc[row, "comment_yn"] = 1
            data.loc[row, "thread_title_yn"] = 0

            row+=1
    
    #Delete NAN rows
    data = data.dropna()
            
    return data

In [17]:
columns = ['subreddit', 'thread_id', 'body', 'comment_yn', 'thread_title_yn' ]

In [22]:
#Running this saves the dataset in your current directory
data = scrape_reddit("losangeles", "anymore", columns)

##### Processing Subreddit : losangeles ####

Threads scraped so far: 0
Count of 'anymore' so far: 0

Threads scraped so far: 20
Count of 'anymore' so far: 3

Threads scraped so far: 40
Count of 'anymore' so far: 5

Threads scraped so far: 60
Count of 'anymore' so far: 6

Threads scraped so far: 80
Count of 'anymore' so far: 8

Threads scraped so far: 100
Count of 'anymore' so far: 10

Threads scraped so far: 120
Count of 'anymore' so far: 14

Threads scraped so far: 140
Count of 'anymore' so far: 15

Threads scraped so far: 160
Count of 'anymore' so far: 15

Threads scraped so far: 180
Count of 'anymore' so far: 16

Threads scraped so far: 200
Count of 'anymore' so far: 18

Threads scraped so far: 220
Count of 'anymore' so far: 19

Threads scraped so far: 240
Count of 'anymore' so far: 23

Threads scraped so far: 260
Count of 'anymore' so far: 23

Threads scraped so far: 280
Count of 'anymore' so far: 28

Threads scraped so far: 300
Count of 'anymore' so far: 32

Threads scraped so fa

## A look at the datafame

In [23]:
data

,subreddit,thread_id,body,comment_yn,thread_title_yn
0,losangeles,4,"I mean, it was. Not so much anymore.",1,0
1,losangeles,16,No one cares about your silly arguments anymor...,1,0
2,losangeles,16,The dipshit NRA shills are out in force today....,1,0
3,losangeles,20,It’s not about needs. We don’t *need* immigran...,1,0
4,losangeles,24,"You got it now, people kill people. We can tal...",1,0
5,losangeles,42,"Now, if we advertised it on Reddit, it wouldn’...",1,0
6,losangeles,71,"""Why can we not build cheap housing in San Ber...",1,0
7,losangeles,79,I fucking can't anymore omlllll,1,0
8,losangeles,93,Sometimes I forget I'm not in an area that get...,1,0
9,losangeles,94,By 2050 America Great Again has liberated the ...,1,0


Let's look at some comments to see how the word "anymore" is being used:

In [ ]:
data[data.thread_id==837].body.values[0]

In [ ]:
data.body[0]

## Add features related to "Anymore"

In [ ]:
#Work In Progress, Don't Panic!